# Table of Content

<a name="outline"></a>

## Setup

- [A](#seca) External Imports
- [B](#secb) Internal Imports
- [C](#secc) Configurations and Paths 
- [D](#secd) Patient Interface and Train/Val/Test Partitioning
- [E](#sece) General Utility Functions


## Training

- [1](#sec1) Training ICE-NODE and The Baselines on MIMIC-III
- [2](#sec2) Training ICE-NODE and The Baselines on MIMIC-IV

<a name="seca"></a>

### A External Imports [^](#outline)

In [1]:
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm

<a name="secb"></a>

### B Internal Imports [^](#outline)

In [2]:
%load_ext autoreload
%autoreload 2

import train as T
import common as C


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [3]:
# HOME and DATA_STORE are arbitrary, change as appropriate.
HOME = os.environ.get('HOME')
DATA_STORE = f'{HOME}/GP/ehr-data'
DATA_FILE = os.path.join(DATA_STORE, 'cprd-data/DUMMY_DATA.csv')
SOURCE_DIR = os.path.abspath("..")

<a name="secd"></a>

### D Configurations and Paths [^](#outline)

**Assign** MIMIC-III and MIMIC-IV directory paths into `mimic3` and `mimic4` variables.

In [4]:
output_dir = 'cprd_artefacts'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [5]:
with C.modified_environ(DATA_FILE=DATA_FILE):
    cprd_dataset = C.datasets['CPRD']
   

In [8]:
"""
optimal hyperparams re: each model.
"""

model_config = {
    'ICE-NODE': f'{SOURCE_DIR}/expt_configs/cprd/icenode_2lr.json' ,
    'ICE-NODE_UNIFORM': f'{SOURCE_DIR}/expt_configs/cprd/icenode_2lr.json' ,
    'GRU': f'{SOURCE_DIR}/expt_configs/cprd/gru.json' ,
    'RETAIN': f'{SOURCE_DIR}/expt_configs/cprd/retain.json',
    'LogReg': f'{SOURCE_DIR}/expt_configs/cprd/window_logreg.json'
}

model_config = {clf: C.load_config(file) for clf, file in model_config.items()}

clfs = ['ICE-NODE', 'ICE-NODE_UNIFORM', 'GRU', 'RETAIN', 'LogReg']

In [9]:
cprd_train_output_dir = {clf: f'{output_dir}/train/{clf}' for clf in clfs}

[Path(d).mkdir(parents=True, exist_ok=True) for d in cprd_train_output_dir.values()]

[None, None, None, None, None]

In [10]:
cprd_reporters = T.make_reporters(clfs, cprd_train_output_dir)

<a name="sece"></a>

### E Patient Interface and Train/Val/Test Patitioning [^](#outline)

In [11]:
code_scheme = {
    'dx': 'dx_cprd_ltc9809',
    'dx_outcome': 'dx_cprd_ltc9809'
}
cprd_interface = C.Subject_JAX.from_dataset(cprd_dataset, code_scheme=code_scheme)
cprd_splits = cprd_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)


In [12]:
cprd_percentiles = cprd_interface.dx_outcome_by_percentiles(20)
cprd_train_percentiles = cprd_interface.dx_outcome_by_percentiles(20, cprd_splits[0])

In [13]:
cprd_models = T.init_models(clfs, model_config, cprd_interface, cprd_splits[0])


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


<a name="sec1"></a>

### 1 Training ICE-NODE and The Baselines on MIMIC-III [^](#outline)

#### ICE-NODE

In [14]:
## TODO: This may take a long time, a pretrained model already exists in (yy).
cprd_trained_icenode = T.train(cprd_models['ICE-NODE'], config=model_config['ICE-NODE'], 
                             splits=cprd_splits, code_groups=cprd_train_percentiles,
                             reporters=cprd_reporters['ICE-NODE'])

  0%|                                                                                                                            | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-p

 37%|██████████████████████████████████████████▏                                                                        | 22/60 [00:43<00:15,  2.41it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|████████████████████████████████████████████                                                                       | 23/60 [00:44<00:15,  2.43it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 40%|██████████████████████████████████████████████                                                                     | 24/60 [00:44<00:14,  2.47it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 42%|████████

 78%|██████████████████████████████████████████████████████████████████████████████████████████                         | 47/60 [00:54<00:05,  2.44it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 80%|████████████████████████████████████████████████████████████████████████████████████████████                       | 48/60 [00:54<00:04,  2.47it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 82%|█████████████████████████████████████████████████████████████████████████████████████████████▉                     | 49/60 [00:54<00:04,  2.43it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 83%|████████

#### ICE-NODE_UNIFORM

In [15]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_icenode_uni = T.train(cprd_models['ICE-NODE_UNIFORM'], config=model_config['ICE-NODE_UNIFORM'], 
                                 splits=cprd_splits, code_groups=cprd_train_percentiles,
                                 reporters=cprd_reporters['ICE-NODE_UNIFORM'])


  0%|                                                                                                                            | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-p

 37%|██████████████████████████████████████████▏                                                                        | 22/60 [00:33<00:12,  2.96it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|████████████████████████████████████████████                                                                       | 23/60 [00:33<00:12,  2.97it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 40%|██████████████████████████████████████████████                                                                     | 24/60 [00:33<00:12,  2.91it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 42%|████████

 78%|██████████████████████████████████████████████████████████████████████████████████████████                         | 47/60 [00:41<00:04,  2.98it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 80%|████████████████████████████████████████████████████████████████████████████████████████████                       | 48/60 [00:41<00:04,  2.98it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 82%|█████████████████████████████████████████████████████████████████████████████████████████████▉                     | 49/60 [00:42<00:03,  2.93it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 83%|████████

#### GRU

In [16]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_gru = T.train(cprd_models['GRU'], config=model_config['GRU'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['GRU'])

  0%|                                                                                                                            | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-p

 37%|██████████████████████████████████████████▏                                                                        | 22/60 [00:08<00:04,  8.01it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|████████████████████████████████████████████                                                                       | 23/60 [00:08<00:04,  8.03it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 40%|██████████████████████████████████████████████                                                                     | 24/60 [00:08<00:04,  7.94it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 42%|████████

 78%|██████████████████████████████████████████████████████████████████████████████████████████                         | 47/60 [00:11<00:01,  7.78it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 80%|████████████████████████████████████████████████████████████████████████████████████████████                       | 48/60 [00:11<00:01,  7.61it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 82%|█████████████████████████████████████████████████████████████████████████████████████████████▉                     | 49/60 [00:11<00:01,  7.70it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 83%|████████

#### RETAIN

In [17]:
## TODO: This can take up to (xx), trained model already exist in (yy).
cprd_trained_retain = T.train(cprd_models['RETAIN'], config=model_config['RETAIN'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['RETAIN'])

  0%|                                                                                                                            | 0/60 [00:00<?, ?it/s]/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-p

 37%|██████████████████████████████████████████▏                                                                        | 22/60 [00:24<00:15,  2.43it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 38%|████████████████████████████████████████████                                                                       | 23/60 [00:25<00:18,  2.03it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 40%|██████████████████████████████████████████████                                                                     | 24/60 [00:25<00:17,  2.09it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 42%|████████

 78%|██████████████████████████████████████████████████████████████████████████████████████████                         | 47/60 [00:35<00:05,  2.58it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 80%|████████████████████████████████████████████████████████████████████████████████████████████                       | 48/60 [00:35<00:04,  2.62it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 82%|█████████████████████████████████████████████████████████████████████████████████████████████▉                     | 49/60 [00:35<00:04,  2.60it/s]/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
 83%|████████

In [18]:
cprd_trained_logreg = T.train(cprd_models['LogReg'], config=model_config['LogReg'], 
                         splits=cprd_splits, code_groups=cprd_train_percentiles,
                         reporters=cprd_reporters['LogReg'])

/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
/home/asem/GP/ICE-NODE/notebooks/../icenode/metric/stat.py:215: RuntimeWarning: invalid value encountered in long_scalars
  rate[k][f'ACC-P{i}-k{k}'] = group_true_positive.sum(
